In [1]:
# Copyright (c) Xi Chen
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

# Borrowed from https://github.com/neocxi/pixelsnail-public and ported it to PyTorch

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [3]:
import gc
import importlib

import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm

from vq_vae_2 import Model
from pixel_snail import PixelSNAIL
from scheduler import CycleScheduler

import wandb

In [4]:
wandb.init()
wandb.run.name = 'PixelSNAIL mid'
wandb.run.save()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: qja1998. Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


True

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
num_split = 100
class MelData(Dataset):
    def __init__(self, file_path):
        self.data = []
        genre = ['classical', 'rock', 'electronic', 'pop', 'deam']
        print("Load data...")
        for g in genre:
            print(f'{g}:')
            for i in tqdm(range(1, 2059)):
                for j in range(num_split):
                    tmp_path = f'{file_path}/{g}/{i}-{j}.csv'
                    try:
                        mel = pd.read_csv(tmp_path).iloc[:, :512]
                        if mel.shape == (80, 512):
                            self.data.append((mel, g, i, j))
                    except FileNotFoundError:
                        continue
            print()
                    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        mel, g, i, j = self.data[idx]
        mel = torch.from_numpy(pd.get_dummies(mel).values)
        mel = mel.type(torch.cuda.FloatTensor)
        
        return (g, i, j), mel

class EmotionalData(Dataset):
    def __init__(self, file_path):
        self.data = pd.read_csv(file_path)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        tmp_data = self.data.iloc[idx]
        genre, idx = tmp_data[0].split('_')
        emo = tmp_data[1:]
        return idx, genre, torch.FloatTensor(emo)

In [7]:
batch_size = 32

In [8]:
EMO_PATH = "./mean_data.csv"
MEL_ARR_PATH = "./split_mel_array"
SAVE_PATH = "./save_models"
mel_arr_data = MelData(MEL_ARR_PATH)
#emo_data = EmotionalData(EMO_PATH)


Load data...
classical:


  0%|          | 0/2058 [00:00<?, ?it/s]


rock:


  0%|          | 0/2058 [00:00<?, ?it/s]


electronic:


  0%|          | 0/2058 [00:00<?, ?it/s]


pop:


  0%|          | 0/2058 [00:00<?, ?it/s]


deam:


  0%|          | 0/2058 [00:00<?, ?it/s]

In [9]:
mel_arr_data_loader = DataLoader(
        dataset=mel_arr_data, batch_size=batch_size)

#emo_data_loader = DataLoader(
#        dataset=emo_data, batch_size=batch_size, shuffle=True)

In [10]:
def scaled(x):
    return (x + 80.0) / (3.8147e-06 + 80)
def unscaled(x):
    return x * (3.8147e-06 + 80) - 80.0

In [11]:
# extract idices
# torch.Size([32, 10, 128]) torch.Size([32, 20, 256])
@torch.no_grad()
def extract_indice(mel_data, model):
    for _, mel in mel_data:
        x = scaled(mel)
        x = x.unsqueeze(1).to(device)
        _, _, _, ids = model.encode(x)
        try:
            ids_t = torch.cat([ids_t, ids[0]], dim=0)
            ids_m = torch.cat([ids_m, ids[1]], dim=0)
            ids_b = torch.cat([ids_b, ids[2]], dim=0)
        except Exception as e:
            print(e)
            ids_t, ids_m, ids_b = ids[0].clone().detach(), ids[1].clone().detach(), ids[2].clone().detach()
    return ids_t, ids_m, ids_b
            

In [12]:
num_hiddens = 128 #128
num_residual_hiddens = 32
num_residual_layers = 4
embedding_dim = 64 #64 : 코드북에서 한 ebedding의 크기
num_embeddings = 256 #512 : embbeding vector 개수 -> indices의 최대 크기
commitment_cost = 0.25

In [13]:
import vq_vae_2_half_tmb
importlib.reload(vq_vae_2_half_tmb)
from vq_vae_2_half_tmb import Model

In [15]:
model = Model(num_hiddens=num_hiddens, 
                  num_residual_layers=num_residual_layers,
                  num_residual_hiddens=num_residual_hiddens,
                  num_embeddings=num_embeddings,
                  embedding_dim=embedding_dim, 
                  commitment_cost=commitment_cost).to(device)

score = 0.008331568911671638
MODEL_PATH = f'{SAVE_PATH}/vqvae2_tmb-{score:.5f}_dict.pt'
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [16]:
ids_t, ids_m, ids_b = extract_indice(mel_arr_data_loader, model)

local variable 'ids_t' referenced before assignment


In [17]:
del mel_arr_data_loader, mel_arr_data

In [18]:
print(ids_t.size(), ids_m.size(), ids_b.size())

torch.Size([18232, 10, 16]) torch.Size([18232, 20, 32]) torch.Size([18232, 40, 64])


In [19]:
gc.collect()

9

In [20]:
def train(hier, epoch, loader, model, optimizer, scheduler, device, batch_size):
    
    loader = tqdm(loader)

    criterion = nn.CrossEntropyLoss()
    
    for i, (top, middle, bottom) in enumerate(loader):
        model.zero_grad()

        top = top.to(device)
        # print(top.shape, bottom.shape)

        if hier == 'top':
            target = top
            out, _ = model(top)
            
        elif hier == 'middle':
            target = middle.to(device) # bottom
            out, _ = model(target, condition=top)
            
        elif hier == 'bottom':
            target = bottom.to(device)
            out, _ = model(target, condition=middle)

        loss = criterion(out, target)
        loss.backward()

        if scheduler is not None:
            scheduler.step()
        optimizer.step()

        _, pred = out.max(1)
        correct = (pred == target).float()
        accuracy = correct.sum() / target.numel()

        lr = optimizer.param_groups[0]['lr']

        wandb.log({
                "Loss": loss,
                "Accuracy": accuracy
        })
        
        loader.set_description(
            (
                f'epoch: {epoch + 1}\t loss: {loss.item():.5f}\t '
                f'acc: {accuracy:.5f}\t lr: {lr:.5f}'
            )
        )

class PixelTransform:
    def __init__(self):
        pass

    def __call__(self, input):
        ar = np.array(input)

        return torch.from_numpy(ar).long()

In [21]:
def train(hier, epoch, loader, model, optimizer, scheduler, device, batch_size):
    
    loader = tqdm(loader)

    criterion = nn.CrossEntropyLoss()
    
    for i, data in enumerate(loader):
        optimizer.zero_grad()
        
        top, middle, bottom = data

        top = top.to(device)
        # print(top.shape, bottom.shape)

        if hier == 'top':
            target = top
            out, _ = model(top)
            
        elif hier == 'middle':
            target = middle.to(device) # bottom
            out, _ = model(target, condition=top)
            
        elif hier == 'bottom':
            target = bottom.to(device)
            out, _ = model(target, condition=middle)

        loss = criterion(out, target)
        loss.backward()

        if scheduler is not None:
            scheduler.step()
        optimizer.step()

        _, pred = out.max(1)
        correct = (pred == target).float()
        accuracy = correct.sum() / target.numel()

        lr = optimizer.param_groups[0]['lr']

        wandb.log({
                "Loss": loss,
                "Accuracy": accuracy
        })
        
        loader.set_description(
            (
                f'epoch: {epoch + 1}\t loss: {loss.item():.5f}\t '
                f'acc: {accuracy:.5f}\t lr: {lr:.5f}'
            )
        )

class PixelTransform:
    def __init__(self):
        pass

    def __call__(self, input):
        ar = np.array(input)

        return torch.from_numpy(ar).long()

In [22]:
epoch = 201
channel = 256
n_res_block = 4
n_res_channel = 256
n_out_res_block = 0
n_cond_res_block = 3
dropout = 0.1

In [23]:
class IDsData(Dataset):
    def __init__(self, ids_t, ids_m, ids_b):
        self.ids_t = ids_t
        self.ids_m = ids_m
        self.ids_b = ids_b
        
    def __len__(self):
        return len(self.ids_t)
    
    def __getitem__(self, idx):
        return self.ids_t[idx], self.ids_m[idx], self.ids_b[idx]

In [24]:
ids_data = IDsData(ids_t, ids_m, ids_b)

In [25]:
import importlib, pixel_snail
importlib.reload(pixel_snail)
from pixel_snail import PixelSNAIL

In [26]:
model_middle = PixelSNAIL(
    [20, 32], # [20, 256]
    256,
    channel,
    5,
    4,
    n_res_block,
    n_res_channel,
    dropout=dropout,
    n_cond_res_block=n_cond_res_block,
    cond_res_channel=n_res_channel,
)

In [27]:
#model_middle.load_state_dict(torch.load(f'{SAVE_PATH}/pixelsnail_ckp/pixelsnail_mid_tmb.pt'))

In [30]:
lr = 1e-4
batch_size = 16
ids_loader = DataLoader(
        ids_data, batch_size=batch_size, shuffle=True, drop_last=True
    )
model_middle = model_middle.to(device)
optimizer_middle = optim.AdamW(model_middle.parameters(), lr=lr)
scheduler_mid = CycleScheduler(optimizer_middle, lr, n_iter=len(ids_loader) * epoch, momentum=None)
for i in range(epoch):
    loss = train('middle', i, ids_loader, model_middle, optimizer_middle, None, device, batch_size)
    torch.save(model_middle.state_dict(), f'{SAVE_PATH}/pixelsnail_ckp/pixelsnail_mid_tmb.pt')


  0%|          | 0/1139 [00:00<?, ?it/s]

  0%|          | 0/1139 [00:00<?, ?it/s]

  0%|          | 0/1139 [00:00<?, ?it/s]

  0%|          | 0/1139 [00:00<?, ?it/s]

  0%|          | 0/1139 [00:00<?, ?it/s]

...

  0%|          | 0/1139 [00:00<?, ?it/s]

  0%|          | 0/1139 [00:00<?, ?it/s]

  0%|          | 0/1139 [00:00<?, ?it/s]

  0%|          | 0/1139 [00:00<?, ?it/s]

  0%|          | 0/1139 [00:00<?, ?it/s]

In [ ]:


lr = 1e-4
batch_size = 8
mel_arr_data_loader = DataLoader(
        dataset=mel_arr_data, batch_size=batch_size)
model_middle = model_middle.to(device)
optimizer_middle = optim.AdamW(model_middle.parameters(), lr=lr)
scheduler_mid = CycleScheduler(optimizer_middle, lr, n_iter=len(mel_arr_data_loader) * epoch, momentum=None)
for i in range(epoch):
    loss = train('middle', i, mel_arr_data_loader, model, model_middle, optimizer_middle, None, device, batch_size)
    torch.save(model_middle.state_dict(), f'{SAVE_PATH}/pixelsnail_ckp/pixelsnail_mid_tmb.pt')


  0%|          | 0/2279 [00:00<?, ?it/s]

  0%|          | 0/2279 [00:00<?, ?it/s]

  0%|          | 0/2279 [00:00<?, ?it/s]

  0%|          | 0/2279 [00:00<?, ?it/s]

  0%|          | 0/2279 [00:00<?, ?it/s]

...

  0%|          | 0/2279 [00:00<?, ?it/s]

  0%|          | 0/2279 [00:00<?, ?it/s]

  0%|          | 0/2279 [00:00<?, ?it/s]

  0%|          | 0/2279 [00:00<?, ?it/s]

KeyboardInterrupt: 